# 5. Passage Retrieval - Dense Embedding

## 강의소개

- 앞선 4강에서는 sparse embedding 기반 문서 검색에 대해 배워보았습니다.
- 이번 5강에서는 앞서 배운 sparse embedding이 가지는 한계점들에 대해 알아보고, 이를 해결할 수 있는 dense embedding에 대해 배워보겠습니다.
- Dense embedding의 개념을 알아보고, 어떻게 dense embedding을 학습하고 문서 검색을 수행할 수 있을지에 대해 실습과 함께 자세히 알아보겠습니다.

<br>

## Further Reading

- [Dense Passage Retrieval for Open-Domain Question Answering](https://arxiv.org/abs/2004.04906)
- [Open domain QA tutorial: Dense retrieval](https://github.com/danqi/acl2020-openqa-tutorial/blob/master/slides/part5-dense-retriever-e2e-training.pdf)

<br>

## Reference

- [https://github.com/danqi/acl2020-openqa-tutorial/tree/master/slides](https://github.com/danqi/acl2020-openqa-tutorial/tree/master/slides)
- [http://jalammar.github.io/illustrated-bert/](http://jalammar.github.io/illustrated-bert/)
- Chen, Danqi, and Wen-tau Yih. "Open-domain question answering." Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics: Tutorial Abstracts. 2020
- [http://www.cs.umd.edu/class/fall2018/cmsc470/slides/slides_12.pdf](http://www.cs.umd.edu/class/fall2018/cmsc470/slides/slides_12.pdf)

<br>

## 5.1 Introduction to Dense Embedding

### 5.1.1 Passage Embedding

- 구절(Passage)을 벡터로 변환하는 것

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1nRGnLJSajUD4fAqS9Nxd-Q3_2y1CZZrc' width=800/>

<br>

### 5.1.2 Sparse Embedding

- TF-IDF 벡터는 Sparse 하다.
  - 많은 벡터값 중 0이 아닌 값은 적다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1nTKSE6nwN_5iJPfe0vRu16urOlIi4TlI' width=800/>

<br>

### 5.1.3 Limitations of sparse embedding

1. 차원의 수가 매우 크다.
  - compressed format으로 극복 가능
2. 유사성을 고려하지 못한다.
  - 두 단어가 유사한 의미를 갖는 단어라고 하더라도 두 단어는 vector space에서 서로 다른 dimension을 차지하는 값이 된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1nW-CW7zn5jk8weQ-SGUbZysgLONqqJI4' width=800/>

<br>

### 5.1.4 Dense Embedding 이란?

- Complementary(상호보완적인) to sparse representations by design

<br>

- 더 작은 차원의 고밀도 벡터 (length = 50 ~ 1000)
  - 1000 이상의 차원을 갖는 경우는 드물다.
- 각 차원이 특정 term에 대응되지 않음
- 대부분의 요소가 non-zero 값

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1nZXp03SfjiBO9w_x2R1RugFEj9Eo2wJD' width=800/>

<br>

### 5.1.5 Retrieval: Sparse vs Dense

- Sparse
  - 단어의 존재유무를 확인하기에는 유용하다.
  - 하지만 의미적으로 확인하기에는 쉽지 않다.
  - 차원이 크기 때문에 알고리즘을 활용하는 데 한계가 있다.
- Dense
  - 의미를 파악할 수 있다.
  - 차원이 비교적 작기 때문에 여러 알고리즘을 활용할 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1nZdXum0Se6Ya0zny70pdQGGHw-hBC2Bc' width=800/>

<br>

- Sparse Embedding
  - 중요한 term들이 정확히 일치해야 하는 경우 성능이 뛰어남
  - 임베딩이 구축되고 나서는 추가적인 학습이 불가능함
- Dense Embedding
  - 단어의 유사성 또는 맥락을 파악해야 하는 경우 성능이 뛰어남
  - 학습을 통해 임베딩을 만들며 추가적인 학습 또한 가능

<br>

- 최근 사전학습 모델의 등장, 검색 기술의 발전 등으로 인해 Dense Embedding을 활발히 이용

<br>

### 5.1.6 Overview of Passage Retrieval with Dense Embedding

- dense embedding을 위한 인코더 모델의 구조는 아래 그림과 같다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1njpRSqec--SinHxFyxHm5AomHRP7ciem' width=600/>

- $\text{BERT}_Q$
  - question에 대한 인코더 모델
  - BERT 모델의 [CLS] 토큰의 벡터값에 대해 $W_q$를 적용하여 질문 문장에 대한 $h_q$라는 인코딩 벡터를 만들어낸다.
- $\text{BERT}_B$
  - passage에 대한 인코더 모델
  - $\text{BERT}_Q$와는 다른 파라미터를 사용하는 BERT 모델
  - BERT 모델의 [CLS] 토큰의 벡터값에 대해 $W_b$를 적용하여 특정 Passage에 대한 $h_b$라는 인코딩 벡터를 만들어낸다.
- 여기서 중요한 점은 $h_q$와 $h_b$는 동일한 차원을 가져야 한다는 점이다.
  - 차원이 동일해야 유사도를 구할 수 있다.
  - 일반적인 유사도 측정 방법은 dot product이다.

<br>

- 훈련 대상 (1)
  - Dense embedding을 생성한 인코더 훈련
  - 두 인코더를 같은 인코더를 사용하는 경우도 있다.
  - 두 인코더의 architecture는 동일하다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1nlmmKQwAYse4DrkhCmrquSVejFqGMtew' width=800/>

<br>

- 질문과 문서를 비교하여 관련 문서 추출

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1nqV931kXhT5FlNTe7iiWwyVmGJjLJ_su' width=800/>

<br>

## 5.2 Training Dense Encoder

### 5.2.1 What can be Dense Encoder?

- BERT와 같은 Pre-trained language model (PLM)이 자주 사용
- 그 외 다양한 nerual network 구조도 가능

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1nuzQuGqrsWprTNwvDAP_wFzWIGi7C04q' width=600/>

- MRC의 경우 input으로 question과 passage를 동시에 넣어주지만 retrieval의 경우에는 question과 passage를 각각의 인코더에 독립적으로 입력으로 사용한다.

<br>

### 5.2.2 BERT as dense encoder

- `[CLS]` token의 output 사용

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1nvhWA46UeaZHO5CtP2hNVOjM8-_yMU_7' width=600/>

<br>

### 5.2.3 Dense Encoder 구조

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1o1pOBc16ZjXL-1xGRP_YHY7tqqme4aC7' width=800/>

- 질문에 대한 정답인 passage의 경우에는 유사도 점수가 높아지도록 학습한다.
- 질문에 대한 정답이 아닌 passage의 경우에는 유사도 점수가 마이너스값이 되도록 학습한다.

<br>

### 5.2.4 Dense Encoder의 학습 목표와 학습 데이터

#### 5.2.4.1 학습 목표 & Challenge

- 학습 목표
  - 연관된 question과 passage dense embedding 간의 거리를 좁히는 것(또는 inner product를 높이는 것)
  - 즉, higher similarity
- Challenge
  - 연관된 question / passage를 어떻게 찾을 것인가? -> 기존 MRC 데이터셋을 활용
  - MRC 데이터셋을 활용하게 되면 기존에 있는 question과 passage pair는 정답 데이터셋이라고 볼 수 있고 그 이외의 passage들은 관련 없는 데이터셋으로 볼 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1o5EnluQemxjcLlvO3m92iVc4TqbKlgII' width=800/>

<br>

#### 5.2.4.2 Negative Sampling

- Positive
  - 연관된 question과 passage 간의 dense embedding 거리를 좁히는 것
  - higher similarity
  - question에 pair한 passage를 사용
- Negative
  - 연관되지 않은 question과 passage 간의 embedding 거리는 멀어야 함
  - question에 대한 passage를 랜덤하게 추출

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1oQtfaPWaibyz0W9mO5xM4-aC9sEwWnHR' width=400/>

<br>

#### 5.2.4.3 Choosing negative examples

1. Corpus 내에서 랜덤하게 뽑기
2. 좀 더 헷갈리는 negative 샘플들 뽑기
  - ex) 높은 TF-IDF 스코어를 가지지만 답을 포함하지 않는 샘플
  - 해당 방법론을 사용하여 정확성을 올린 케이스가 있다.

<br>

### 5.2.5 Objective function

- Positive passage에 대한 negative log likelihood(NLL) loss 사용
- Positive passage의 score를 확률화하기 위해서 Positive passage와 question 간의 similarity score와 negative sample에 대한 similarity score를 softmax하여 구해진 확률값을 NLL를 적용하여 학습하는 방법을 사용한다.
  - NLL of positive passage 확인

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1oaiAAHRocGMLu5OUsINySOG-qBXkKeF5' width=800/>

<br>

### 5.2.6 Evaluation Metric for Dense Encoder

- Top-k retrieval accuracy
  - retrieve된 passage 중에 답(MRC의 answer)을 포함하는 passage의 비율

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1otSdd5tq_q01HUMZM1lc4OKaYVfn7WCX' width=500/>


<br>

## 5.3 Passage Retrieval with Dense Encoder

### 5.3.1 From dense encoding to retrieval

- Inference
  - Passage와 query를 각각 embedding 한 후, query로부터 가까운 순서대로 passage의 순위를 매김
- Passage에 대한 오프라인으로 한 번 진행
- Question이 들어올 때 마다 GPU에 저장된 passage vector와 question vector를 inner product와 같은 방법을 통해 거리를 측정하여 가장 유사도가 높은 passage를 반환한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1ovEAn8_NOKGet-ccMEUT2SC-rovZUVQ6' width=800/>

<br>

### 5.3.2 From retrieval to open-domain question answering

- Retriever를 통해 찾아낸 Passage을 활용하여 MRC 모델로 답을 찾음

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1p2dOU9y-Pfh1iQcwoR30XagQwPIQTfE3' width=800/>

<br>

### 5.3.3 How to make better dense encoding

- 학습 방법 개선 (e.g., DPR)
- 인코더 모델 개선 (BERT보다 큰, 정확한 Pre-trained 모델)
- 데이터 개선 (더 많은 데이터, 전처리 등)